In [2]:
# !pip install aerospike==4.0.0

     |████████████████████████████████| 4.0 MB 2.5 MB/s eta 0:00:01


In [ ]:
# docker run -d -p 3000-3002:3000-3002 aerospike:ce-5.3.0.16

In [20]:
import aerospike

write_policy = {"key": aerospike.POLICY_KEY_SEND}
policies = {"write": write_policy, "total_timeout": 3000}
# Thiết lập kết nối
config = {
    'hosts': [('localhost', 3000)],  # Điền thông tin kết nối đến máy chủ Aerospike của bạn
    # "policies": policies
}

client = aerospike.client(config).connect()



In [4]:
namespace = "test"
set = "long"
ttl = 120 # ttl in s

In [22]:
def put(key, value):
    key_ = (namespace, set, str(key))
    try:
        # client.put(key_, value, meta={"ttl": ttl}, policy= policies)  # policies keysend ở đây k ăn (dẫn chứng là ở con userprofile k hiện )
        client.put(key_, value, meta={"ttl": ttl})
    except Exception as e:
        print("Key error ", key)
        print("Value error ", e)

In [23]:
data = {"name":"long"}

put("12345", data)

In [2]:
def get(key):
        key = (namespace, set, key)
        try:
            (key, metadata, record) = client.get(key)

        except Exception as e: 
            print("read error ", e)
            
        return record

# key: ns, set, key (send_key thi moi thay), digest -bytearray (muc dich la bam cai key ra cho bang nhau)
# metadata: ttl (moi 1 lan get set dau capj nhat so nay) , gen(thay doi bn lan)
# record: dict {"bin":"val"}

# vi dụ 
# ('test', 'long', None, bytearray(b'\xd9\xf9]`\x18\x12\x0eh\xec\x02\xcc\x19\x80\x94C,\x9f\x8e\x05\xa5'))
# {'ttl': 116, 'gen': 1}
# {'name': 'long'}

In [10]:
get("12345")

('test', 'long', None, bytearray(b'\xd9\xf9]`\x18\x12\x0eh\xec\x02\xcc\x19\x80\x94C,\x9f\x8e\x05\xa5'))
{'ttl': 116, 'gen': 1}
{'name': 'long'}


{'name': 'long'}

In [16]:
def delete(key):
        """
        Delete row by key.
        """
        key = (namespace, set, key)
        try:
            client.remove(key)
            print("Delete successfully")
        #             LOGGER.info("Successful delete!")
        except Exception as e:
            print("Error: {}".format(e))

In [21]:
delete("12345")

Delete successfully


In [ ]:
scan = client.scan(namespace, set)

b= []
def process_record(a):
    b.append(a) 
    
scan_opts = {"concurrent": True, "nobins": True, "percent":1}  # chu y cai percent này, nếu k quét set lớn sẽ treo client

scan.foreach(process_record, options=scan_opts)